<a href="https://colab.research.google.com/github/NINGTANG1124/UPF-HFI/blob/main/notebooks/modeling-%E4%B8%AD%E6%96%87%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# connect to googledrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 合并 HFI 到建模数据表

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 120)

# 读取
survey_df = pd.read_excel( "/content/drive/MyDrive/UPF-HFI/Model/outcome/survey_with_HFI.xlsx")
df_upf    = pd.read_excel("/content/drive/MyDrive/UPF-HFI/Model/outcome/weighted_upf_percent.xlsx")

In [3]:
# ==== 2. 先检查 ====
issues = []

# 检查关键列
for col in ["UserID_clean", "HFI_binary", "HFI_raw_score"]:
    if col not in survey_df.columns:
        issues.append(f"survey 缺少 {col}")

for col in ["UserID_clean", "weighted_upf_percent"]:
    if col not in df_upf.columns:
        issues.append(f"upf 缺少 {col}")

# 检查 ID 是否有缺失/重复
for name, df in [("survey", survey_df), ("upf", df_upf)]:
    n_na = df["UserID_clean"].isna().sum()
    n_dup = df["UserID_clean"].duplicated().sum()
    if n_na:  issues.append(f"{name} 的 UserID_clean 有 {n_na} 个缺失")
    if n_dup: issues.append(f"{name} 的 UserID_clean 有 {n_dup} 个重复")

# 检查 upf_percent 是否数值
bad_upf = pd.to_numeric(df_upf["weighted_upf_percent"], errors="coerce").isna().sum()
if bad_upf:
    issues.append(f"weighted_upf_percent 有 {bad_upf} 个非数值/缺失")

# 打印检查结果
if issues:
    print("检查发现以下问题：")
    for i, msg in enumerate(issues, 1):
        print(f"{i}. {msg}")
else:
    print("没有发现问题，可以直接处理")


没有发现问题，可以直接处理


In [4]:
# ==== 检查关键变量缺失情况 ====
check_cols = ["HFI_binary", "weighted_upf_percent"]
missing_info = {}

for col in check_cols:
    if col not in survey_df.columns and col not in df_upf.columns:
        print(f"数据中找不到列: {col}")
        continue

    if col in survey_df.columns:
        n_miss = survey_df[col].isna().sum()
        missing_info[col] = n_miss
        print(f"{col} 在 survey_df 中缺失 {n_miss} 行")

    if col in df_upf.columns:
        n_miss = df_upf[col].isna().sum()
        missing_info[col] = n_miss
        print(f"{col} 在 upf_df 中缺失 {n_miss} 行")

# ==== 判断是否需要处理 ====
if all(v == 0 for v in missing_info.values()):
    print("两列都没有缺失，可以直接进入合并/建模")
else:
    print("存在缺失，需要处理后再合并")


HFI_binary 在 survey_df 中缺失 18 行
weighted_upf_percent 在 upf_df 中缺失 0 行
存在缺失，需要处理后再合并


In [5]:
# 1. 合并 survey 和 upf
df_model = pd.merge(survey_df, df_upf, on="UserID_clean", how="inner")

# 2. 删掉 HFI_binary 缺失的
df_model = df_model.dropna(subset=["HFI_binary"])

print("合并后样本量：", len(df_model))


合并后样本量： 308


In [6]:
# 导出合并后的数据
df_model.to_excel("/content/drive/MyDrive/UPF-HFI/Model/outcome/merged_model.xlsx", index=False)

print("已导出到 /content/drive/MyDrive/UPF-HFI/Model/outcome/merged_model.xlsx")


已导出到 /content/drive/MyDrive/UPF-HFI/Model/outcome/merged_model.xlsx


# 建模部分

## 步骤 0：一次性 dropna（主分析用的完整案例集）

In [17]:
# 1) 统一主分析变量清单（一次性 dropna）
all_vars = [
    "weighted_upf_percent","HFI_binary","income",
    "gender_participant","ethn_participant","age_participant",
    "child_numbers", "employ"
]
df_all = df_model[all_vars].apply(pd.to_numeric, errors="ignore").dropna().copy()
print("主分析样本量:", len(df_all))

# 这次是对所有 all_vars 里的变量做 完全案例（complete case）筛选，也就是任何一个变量缺失都会被剔除。因为 income、age_participant 等变量里可能有缺失，所以样本量从 308 → 305。

主分析样本量: 299


/tmp/ipython-input-2805321717.py:7: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_all = df_model[all_vars].apply(pd.to_numeric, errors="ignore").dropna().copy()


In [18]:
import pandas as pd

all_vars = ["weighted_upf_percent","HFI_binary","income",
    "gender_participant","ethn_participant","age_participant",
    "child_numbers", "employ"]

# 哪些列在 df_model 里缺失了多少？
print("每列缺失数：")
print(df_model[all_vars].isna().sum())

# 标出完全案例掩码
mask_complete = df_model[all_vars].notna().all(axis=1)

# 被 drop 的行 + 缺失列
dropped = df_model.loc[~mask_complete, ["UserID_clean"] + all_vars].copy()
dropped["missing_cols"] = dropped.apply(
    lambda r: [c for c in all_vars if pd.isna(r[c])], axis=1
)
print("\n被剔除的样本（以及缺失了哪些列）：")
print(dropped[["UserID_clean","missing_cols"]])


每列缺失数：
weighted_upf_percent    0
HFI_binary              0
income                  3
gender_participant      0
ethn_participant        0
age_participant         0
child_numbers           4
employ                  2
dtype: int64

被剔除的样本（以及缺失了哪些列）：
    UserID_clean     missing_cols
21        BFD041         [income]
38        BFD061         [employ]
95        BFD124  [child_numbers]
115       BFD151         [employ]
173       BFD220  [child_numbers]
189       BFD243         [income]
230       BFD295         [income]
272       BFD353  [child_numbers]
285       BFD371  [child_numbers]


## 步骤 1：检查“内部捣乱”变量（多重共线性 / VIF）

In [19]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

# One-hot 分类变量（drop_first避免虚拟变量陷阱）
vif_df = pd.get_dummies(df_all.drop(columns=["weighted_upf_percent"]), drop_first=True)

vif_results = pd.DataFrame({
    "Variable": vif_df.columns,
    "VIF": [variance_inflation_factor(vif_df.values, i) for i in range(vif_df.shape[1])]
}).sort_values("VIF", ascending=False)

print(vif_results)


             Variable       VIF
2  gender_participant  6.819030
4     age_participant  6.167936
5       child_numbers  5.821079
1              income  5.279584
3    ethn_participant  2.635621
6              employ  2.376755
0          HFI_binary  1.742598


你现在跑 VIF 检查的就是主分析用的那几个变量（df_all 里的），这样做是对的，因为 BMJ 主分析也是保证同一批自变量在五个模型里样本一致、可比性强。
如果你把所有可能变量都放进去跑 VIF，

一方面会混进不参与主分析的变量，
另一方面会因为缺失值多导致样本量骤减，不符合主分析的可比性原则。
所以VIF 检查只需要针对你要进 M1–M5 模型的自变量就可以，不用所有变量都跑。

所以你这个结果意味着：
没有达到“强共线性”的红线（10），可以保留全部变量
但 gender、age、income 三个变量有一定共线性，如果你做敏感性分析，可以尝试单独去掉其中一个，看结果是否稳定
换句话说，现在是安全可用的，直接进入 M1–M5 分析没问题。

## 步骤 2：主分析

In [27]:
print(df_all.columns.tolist())   # 应该包含 HFI_binary, income, gender_participant, ethn_participant, age_participant, child_numbers, employ, weighted_upf_percent


['weighted_upf_percent', 'HFI_binary', 'income', 'gender_participant', 'ethn_participant', 'age_participant', 'child_numbers', 'employ']


In [28]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

# 一眼确认下列名是否都在
# print(df_all.columns.tolist())

forms_main = {
    "M1": "weighted_upf_percent ~ HFI_binary",
    "M2": "weighted_upf_percent ~ HFI_binary + income",
    "M3": "weighted_upf_percent ~ HFI_binary + income + C(gender_participant) + C(ethn_participant, Treatment(reference=1))",
    "M4": "weighted_upf_percent ~ HFI_binary + income + C(gender_participant) + C(ethn_participant, Treatment(reference=1)) + age_participant",
    "M5": "weighted_upf_percent ~ HFI_binary + income + C(gender_participant) + C(ethn_participant, Treatment(reference=1)) + age_participant + child_numbers",
    "M6": "weighted_upf_percent ~ HFI_binary + income + C(gender_participant) + C(ethn_participant, Treatment(reference=1)) + age_participant + child_numbers + C(employ)"
}

res = {k: smf.ols(v, data=df_all).fit(cov_type="HC3") for k,v in forms_main.items()}

print("Ns:", {k:int(m.nobs) for k,m in res.items()})  # 应该都一致


Ns: {'M1': 299, 'M2': 299, 'M3': 299, 'M4': 299, 'M5': 299, 'M6': 299}


In [29]:
rows=[]
for k in ["M1","M2","M3","M4","M5","M6"]:
    m = res[k]
    b  = m.params["HFI_binary"]
    lo, hi = m.conf_int().loc["HFI_binary"]
    p  = m.pvalues["HFI_binary"]
    rows.append([k, b, lo, hi, p, int(m.nobs), m.rsquared])

main_tbl = (pd.DataFrame(rows, columns=["Model","HFI_coef_pp","CI_low","CI_high","p","N","R2"])
              .round({"HFI_coef_pp":2,"CI_low":2,"CI_high":2,"p":4,"R2":3}))
print(main_tbl)
# main_tbl.to_excel("out_main_M1_M6.xlsx", index=False)


  Model  HFI_coef_pp  CI_low  CI_high       p    N     R2
0    M1         2.27   -0.67     5.20  0.1299  299  0.008
1    M2         2.57   -0.64     5.79  0.1168  299  0.008
2    M3         3.37    0.15     6.60  0.0403  299  0.077
3    M4         3.25    0.03     6.47  0.0478  299  0.090
4    M5         3.06   -0.23     6.34  0.0681  299  0.094
5    M6         3.32   -0.00     6.64  0.0503  299  0.123


## M7：交互（换成可读标签，便于解释与作图）

In [34]:
import re
import numpy as np
import pandas as pd

# 你的映射表
ethn_map = {
  1:"White British", 2:"Other white group", 3:"Pakistani", 4:"Indian",
  5:"Bangladeshi", 6:"Other Asian group", 7:"Mixed/multiple ethnicity",
  8:"Black / Black British / Black Caribbean / Black African",
  9:"Other ethnic group"
}

effects = []
b0  = m7.params["HFI_binary"]
v00 = m7.cov_params().loc["HFI_binary","HFI_binary"]
se0 = float(np.sqrt(v00))
effects.append(dict(eth_group="White British (ref)", effect=b0, lo=b0-1.96*se0, hi=b0+1.96*se0))

for term in m7.params.index:
    # 既兼容 ethn_name 也兼容数值编码
    if term.startswith("HFI_binary:C("):
        # 抓出 T.后面的内容
        m = re.search(r"T\.(.+)\]$", term)
        if not m:
            continue
        raw = m.group(1)  # 可能是 'Pakistani' 或 '2'

        # 如果是数字就映射成中文名/英文名，否则直接用标签
        try:
            label = ethn_map[int(raw)]
        except ValueError:
            label = raw

        b  = b0 + m7.params[term]
        v  = v00 + m7.cov_params().loc[term,term] + 2*m7.cov_params().loc["HFI_binary", term]
        se = float(np.sqrt(v))
        effects.append(dict(eth_group=label, effect=b, lo=b-1.96*se, hi=b+1.96*se))

effects_df = pd.DataFrame(effects).round(2)
print(effects_df)
# effects_df.to_excel("out_M7_eth_effects.xlsx", index=False)


                                           eth_group  effect      lo      hi
0                                White British (ref)    1.73   -2.95    6.41
1                                        Bangladeshi   -1.73  -18.24   14.77
2  Black / Black British / Black Caribbean / Blac...    9.53    4.87   14.19
3                                             Indian   10.95  -11.86   33.76
4                           Mixed/multiple ethnicity   10.46    0.66   20.25
5                                  Other Asian group    6.33    0.35   12.31
6                                 Other ethnic group   18.34 -228.06  264.73
7                                  Other white group   11.65  -72.34   95.65
8                                          Pakistani    1.44   -4.87    7.75


# 敏感性分析

In [22]:
import numpy as np, pandas as pd

rows = []
for name, m in results.items():
    b = m.params.get("HFI_binary", np.nan)
    ci_l, ci_h = m.conf_int().loc["HFI_binary"]
    p = m.pvalues["HFI_binary"]
    rows.append([name, b, ci_l, ci_h, p, int(m.nobs), m.rsquared])

main_tbl = pd.DataFrame(rows, columns=["Model","coef","CI_low","CI_high","p","N","R2"]) \
             .round({"coef":2,"CI_low":2,"CI_high":2,"p":4,"R2":3})
print(main_tbl)
# main_tbl.to_excel("main_results_M1_M5.xlsx", index=False)


  Model  coef  CI_low  CI_high       p    N     R2
0    M1  2.27   -0.67     5.20  0.1299  299  0.008
1    M2  2.57   -0.64     5.79  0.1168  299  0.008
2    M3  2.99   -0.27     6.24  0.0723  299  0.028
3    M4  2.81   -0.44     6.06  0.0900  299  0.042
4    M5  3.18   -0.16     6.51  0.0617  299  0.069


# 可视化